In [1]:
import sys 
sys.path.append("./modules/")
from markov import *
from mva import *
from simulation import *
import platform

In [2]:
# variables

SystemParameters.alpha = 0.8
SystemParameters.beta = 0.2
SystemParameters.u1 = 15
SystemParameters.u2 = 75
SystemParameters.thinkTime = 5000
SystemParameters.timeSlice = 3
SystemParameters.numClients = 3
SystemParameters.Sio1 = 40
SystemParameters.Sio2 = 180
SystemParameters.qio1 = 0.65 #route to io1
SystemParameters.qio2 = 0.25 # route to io2
SystemParameters.qoutd = 0.1*0.4 #go to delay station
SystemParameters.qouts = 0.1*0.6 # renter the system

In [3]:
matrix = np.array([
    [0,1,0,0,0],
    [0,0,1,0,0],
    [0.004,0.006,0.9,0.065,0.025],
    [0,0,1,0,0],
    [0,0,1,0,0]],np.dtype('d'))


mva = MVA(matrix,[5000,0,2.7,40,180],[StationType.Delay,StationType.LoadIndependent,StationType.LoadIndependent,StationType.LoadIndependent,StationType.LoadIndependent],30)


def produce_graph_and_save(dataframe: pd.DataFrame, name:str, yaxis:str, log:bool = False):
    fig = plt.line(dataframe,y=["CPU","SWAP_IN","IO1","IO2"], x="N",title=name,log_y=log)
    fig.update_yaxes(title_text=yaxis)
    data = fig.to_image("png")
    f = open("./Images/{}.png".format(name),"+wb")
    f.write(data)
    f.close()
    fig.show()
pass

mva()

utilization = mvaToDataframe(mva.utilizations)
throughputs = mvaToDataframe(mva.throughputs)
meanWaits = mvaToDataframe(mva.meanwaits)
meanClients = mvaToDataframe(mva.meanclients)


In [4]:
produce_graph_and_save(utilization,"Utilizations_markov","B/T")

In [5]:
produce_graph_and_save(meanWaits,"meanWaits_markov","ms",True)

In [6]:
produce_graph_and_save(meanClients,"meanClients_markov","n")

In [7]:
produce_graph_and_save(throughputs,"throughputs_markov","C/T")

legend (Delay_station,CPU.Stage,IO1,IO2)


In [8]:
commander = SimulatorCommander()
markov20 = commander.fetch_data(10,"Markov_20")
markov20_negexp = commander.fetch_data(10,"Markov_20_NegExp")

In [9]:
SystemParameters.numClients = 20
markov20["Expected"] = [execute_markov()[1]] * len(markov20)
SystemParameters.u1 = 27
SystemParameters.u2 = 27
SystemParameters.alpha = 0.5
SystemParameters.beta= 0.5 
markov20_negexp["Expected"]= [execute_markov()[1]]*len(markov20)


4.273902312169617e-14
0.9999999999999979
Len  3311
Ndelay 4.431233117775969 Expected 4.438722599034795
Ncpu 1.8032526959811719 Expected 1.4842718857968777
Nio1 1.522510078710564 Expected 1.3563025143771892
Nio2 12.243004107532284 Expected 12.720703000791138
3.3492440354868485e-14
0.9999999999999996
Len  3311
Ndelay 4.4387225219717426 Expected 4.438722599034795
Ncpu 1.4842719266841602 Expected 1.4842718857968777
Nio1 1.3563025397440056 Expected 1.3563025143771892
Nio2 12.720703011600081 Expected 12.720703000791138


In [10]:
def print_cpu_measure(data: pd.DataFrame, scenario, measure):
    view = data[(data["Station"] == "CPU")& (data["Measure"] == measure)]
    fig = plt.line(view, x="Simulation" , y=["LB","HB","R","Expected"], title="{} CPU scenario {}".format(measure,scenario))
    return fig,view
    pass

fig,view = print_cpu_measure(markov20,"Markov","meanclients")
fig.show()

In [11]:

fig,view = print_cpu_measure(markov20,"Markov","meanwaits")
fig.show()

In [12]:
fig,view = print_cpu_measure(markov20_negexp,"Markov_negexp","meanclients")
fig.show()